# South Asia GDP Prediction

This jupyter notebook is developed to predict Gross Domestic Product (GDP) of several countries in the South Asian region by utilizing the nighttime light satellite data. GDP is an important economic indicator that is closely related to the economic development of a country. Therefore, accurately predicting GDP can provide valuable insights into the economic growth of a region.

Nighttime light data provides a unique perspective into human activities and infrastructure, which can be used to estimate electricity consumption, economic activity, and urbanization trends. In this project, I will explore the relationship between nighttime light data and GDP to predict GDP for several countries in South Asia.

In [1]:
import ee
import geemap
import geemap as gee
import json
import requests
import os
import shapefile
import geopandas as gpd

import numpy as np

# Initialize the Earth Engine API
ee.Initialize()

The `SAsia_Merged.shp` is a vector file that was created by combining the countries of Sri Lanka, India, Pakistan, Nepal, Bhutan, and Bangladesh, which are the countries interested in for predicting GDP. To filter data from Google Earth Engine (GEE), it needs to define an area of interest (AOI) that covers the regions of these countries. Instead of using the geometry of `SAsia_Merged.shp` directly, which contains a large number of nodes making it difficult for GEE to filter data efficiently, I can create a square shape that covers the AOI.

To create a square shape that covers the AOI, I first calculated the maximum and minimum coordinates of x (latitudes) and y (longitudes) for the `SAsia_Merged.shp`. I then used these values to create a bounding box that defines square shape. The bounding box is a rectangular box with sides aligned with the x and y axes, created by specifying the minimum and maximum values of both the x and y dimensions. The use of a bounding box allows me to efficiently filter the data from GEE, allowing for faster processing times

In [2]:
vector_file_path = r"sAsia/SAsia_Merged.shp"
vector_data = gpd.read_file(vector_file_path)
vector_bound_coordinates = vector_data.bounds
minX, minY, maxX, maxY = vector_bound_coordinates.values[0][0], vector_bound_coordinates.values[0][1], vector_bound_coordinates.values[0][2], vector_bound_coordinates.values[0][3]

for item in vector_bound_coordinates.values:
    if minX > item[0]: minX = item[0]
    if minY > item[1]: minY = item[1]
    if maxX < item[2]: maxX = item[2]
    if maxY < item[3]: maxY = item[3]

area_of_interest = ee.Geometry.Rectangle([[minX-1, minY-1],[maxX+1, maxY+1]])


To visually display and validate if the created rectangular area covers my AOI, I can create an interactive map using geemap. The map will show the blue area covered by the rectangle and the red polygons that represent the country areas obtained from the GEE feature collection USDOS/LSIB_SIMPLE/2017.

The purpose of this map is to provide a visual representation of the selected rectangular area and to ensure that it encompasses the countries of Sri Lanka, India, Pakistan, Nepal, Bhutan, and Bangladesh. The red polygons represent the actual country boundaries, providing a frame of reference to ensure that our rectangular selection aligns with the desired area of interest.

By using this map, I can visually confirm that the rectangular area I have selected includes the targeted countries and exclude any unwanted areas. This process helps ensure that the data we collect from GEE is accurate and relevant for my analysis.

In [3]:
Map = geemap.Map(zoom=1, lite_mode=False)
countries_interested = ["Pakistan", "Nepal", "Sri Lanka", "India", "Bhutan", "Bangladesh"]
area_of_interest = ee.Geometry.Rectangle([[60, 37],[98, 5]])

Map.addLayer(area_of_interest, {"color":"blue"}, "Rect 1")
Map.center_object(area_of_interest)

for country in countries_interested:

    ca_geom = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq('country_na', country)).geometry()
    Map.addLayer(ca_geom, {"color":"red"}, country)

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

GEE provides access to two night light emission image collections, which were collected by sensors mounted on two different satellites. The first collection is the `VIIRS` (Visible Infrared Imaging Radiometer Suite) satellite night light emission image collection, with data available from January 1st, 1992 to January 1st, 2014. The second collection is the `CCNL` (Climate Change Initiative Combined Nighttime Lights) satellite night light emission image collection, with data available from January 1st, 2014 to June 1st, 2022. Each collection has a different range of minimum and maximum values. This means that the night light data from the two collections cannot be directly compared without first normalizing them into a consistent range.

In [4]:
VIIRS = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMCFG') 
CCNL = ee.ImageCollection('BNU/FGS/CCNL/v1')

To refine our dataset further, I have filtered the GEE night light emission image collections for each year and extracted the cloud-free coverage from VIIRS and band 1 from CCNL.

In [5]:
print("Filtering...")

dataset_ccnl_filtered = {}
dataset_viirs_filtered = {}
get_ccnl = True
get_viirs = False

if get_ccnl:
    print("CCNL", end=' ')
    for year in range(1992,2014):
        print(year ,end=' ')
        ntldata = CCNL.filterBounds(area_of_interest).filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31'))
        ntldata = ntldata.select('b1').mean().clip(area_of_interest)
        dataset_ccnl_filtered[year] = ntldata

if get_viirs:
    print("VIIRS", end=' ')
    for year in range(2014, 2023):
        print(year ,end=' ')
        ntldata = VIIRS.filterBounds(area_of_interest).filter(ee.Filter.date(str(year) + '-01-01', str(year) + '-12-31'))
        ntldata = ntldata.select('cf_cvg').mean().clip(area_of_interest)
        dataset_viirs_filtered[year] = ntldata

Filtering...
CCNL 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 

In [6]:
# For testing
url = 'https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png'

Map = geemap.Map(zoom=8, lite_mode=False)
Map.add_tile_layer(url, name='AlidadeSmoothDark', attribution='&copy; <a href="https://stadiamaps.com/">Stadia Maps</a>, &copy; <a href="https://openmaptiles.org/">OpenMapTiles</a> &copy; <a href="http://openstreetmap.org">OpenStreetMap</a> contributors')
attributes = {'palette': ['000004', '160b39', '400a67', '69166e', '902568', 'd94d3d', 'f1711f', 'fb9d07', 'f8cd37', 'fcffa4']}

for year, data in dataset_ccnl_filtered.items():
    Map.addLayer(data, attributes, str(year) + " CCNL")
    break

for year, data in dataset_viirs_filtered.items():
    Map.addLayer(data, attributes, str(year) + " VIIRS")
    break

Map.center_object(area_of_interest)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

One way to export filtered data sets to Google Drive is by using the following code to export them as GeoTiff files:
```python

for year, data in dataset.items():
    geemap.ee_export_image_to_drive(data, fileNamePrefix="nle_" + str(year), region=area_of_interest, folder='export_nle')
```

However, exporting large GeoTiff files can be time-consuming. An alternative approach to this is to export the data sets as numpy arrays, which is a faster process.

In [7]:
output_folder = r"output"
if not os.path.isdir(output_folder):
    os.mkdir(output_folder)

for country in countries_interested:
    print(country)
    for year, data in dataset_ccnl_filtered.items():
        print(year, end=' ')
        
        data = data.clip(ca_geom)
        
        dl_para = {
            'bands': ['b1'],
            'region': ca_geom,
            'scale': 2000,
            'format': 'NPY'
        }
        print(data.getDownloadUrl(dl_para))
        response = requests.get(data.getDownloadUrl(dl_para))
        _output_ = output_folder + "\\" + country + "_" + str(year) + ".npy"
        open(_output_, "wb").write(response.content)
        # break
    # break

1992 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/f025111a29b11a5cb9951923f301d81b-d68de3bf4e754197c6f2200e3f77ae3e:getPixels
1993 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/8c1fa07b4c522ed1cf147a8284a9c7a7-ab9da946d6116cb0347660041ef34fa0:getPixels
1994 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/badbd3edd713b36c837137e867ec9660-57e83e7e711ec9d9ff4b69935fc69891:getPixels
1995 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/a0374d14f6fb0924679a05e96eacb368-f13e3356ac138da4a5e9787f9d0b4e65:getPixels
1996 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/07fb0101e441dba5c2fc44676059b95d-c456680686a0630c8e178b181dc7912c:getPixels
1997 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/e19847fccd293baa0f94961ad822b77b-8a214a2c3ad24541cba13979fef558d9:getPixels
1998 https://earthengi

1999 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/decf7a354293495d5a84b299e0deaadf-fd5aca3a0845f688db3ec47faa037d1f:getPixels
2000 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/169175bd924e2101c5f0153e1cd3f810-4158f9fbc5a9bd2564fbb1faae613b76:getPixels
2001 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/333db51cef0a89d54e355f7f0ddfef0a-12f4a622381b9604fa3210e034ea9492:getPixels
2002 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/9088e632ff98ae554f6be77ebd1ea270-7cb9c33fb1f35a08ee502c8f72082b4f:getPixels
2003 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/0446771db7ddce5d465026e864454f1e-7a64234442e87690d44afd5ad2466d64:getPixels
2004 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/241b47c2ca4dccd49128dedea453cc0b-03862d9ee006b91f3e6e6fc88e7acbba:getPixels
2005 https://earthengi

2006 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/607980ba63170da7bac6ef189eeef821-62f1b92cff4bc568c8e8c5d482f978fe:getPixels
2007 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/234e9bc72a6719996d38dc3ff3d91a40-070b976a637aa0862a81f0ee01d17832:getPixels
2008 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d27f45becb51a4224dd0054354f8c607-6a164a9261bb1354ebe032bed4a16b4f:getPixels
2009 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/d6f961c4a862a86e2fff2fa8d218568b-73ba7213f3cddfc7e282d52ee4eea667:getPixels
2010 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6088dbf516ddb4b8ef473e916ae76f9d-bb375ae1d822d98fafe381e5d75e4ad2:getPixels
2011 https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/027adbc949cca2652b932085a831dc8b-5a1939adee8884595f428c3bee769339:getPixels
2012 https://earthengi